In [20]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib

In [21]:
df = pd.read_csv("dataset/output_file.csv")
df['sex'] = df['sex'].map({'female': 0, 'male': 1})

In [22]:
df

,age,height,sex,days_to_goal,activity_level,experience,Exercise
0,56,178,0,108,4,4,"Hammer Curls, Chest Flyes, Seated Rows, Overhe..."
1,35,178,1,109,1,1,"Hammer Curls, Chest Flyes, Seated Rows, Overhe..."
2,43,172,1,143,4,5,"Hammer Curls, Chest Flyes, Seated Rows, Overhe..."
3,25,185,1,130,4,3,"Hammer Curls, Chest Flyes, Seated Rows, Overhe..."
4,57,170,1,139,2,3,"Hammer Curls, Chest Flyes, Seated Rows, Overhe..."
...,...,...,...,...,...,...,...
995,20,174,0,122,1,3,"Leg Press, Russian Twists, Face Pulls, 3 sets ..."
996,40,175,0,123,4,1,"Leg Press, Russian Twists, Face Pulls, 3 sets ..."
997,51,185,0,134,5,4,"Leg Press, Russian Twists, Face Pulls, 3 sets ..."
998,38,171,0,138,4,5,"Leg Press, Russian Twists, Face Pulls, 3 sets ..."


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             1000 non-null   int64 
 1   height          1000 non-null   int64 
 2   sex             1000 non-null   int64 
 3   days_to_goal    1000 non-null   int64 
 4   activity_level  1000 non-null   int64 
 5   experience      1000 non-null   int64 
 6   Exercise        1000 non-null   object
dtypes: int64(6), object(1)
memory usage: 54.8+ KB


In [24]:
# Assuming your DataFrame is named 'df'
df['Exercise'] = df['Exercise'].astype('category')

# Separate features (X) and target variable (y)
X = df.drop('Exercise', axis=1)
y = df['Exercise']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for numerical and categorical features
numeric_features = X.select_dtypes(include=['int64']).columns
numeric_transformer = StandardScaler()

categorical_features = X.select_dtypes(include=['category']).columns
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and RandomForestClassifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', RandomForestClassifier(random_state=42))])

# Define hyperparameters to tune
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=8, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters found by the grid search
print("Best Parameters:")
print(grid_search.best_params_)

# Save the trained model to a file
joblib.dump(grid_search, 'trained_model.joblib')

# Make predictions on the test set
y_pred = grid_search.predict(X_test)


Best Parameters:
{'classifier__max_depth': 10, 'classifier__n_estimators': 100}


In [26]:

new_data = pd.DataFrame({
    'age': [30],
    'height': [175],
    'sex': [1],
    'days_to_goal': [10],
    'activity_level': [3],
    'experience': [2],
})

# Display the new data
print("New Data:")
print(new_data)


New Data:
   age  height  sex  days_to_goal  activity_level  experience
0   30     175    1            10               3           2


In [27]:
# Load the saved model
loaded_model = joblib.load('trained_model.joblib')

# Assuming you have new_data for prediction
new_data_predictions = loaded_model.predict(new_data)


In [28]:
new_data_predictions

array(['Arnold Press, Leg Curls, Incline Press, 4 sets (12-10-8-6 reps)'],
      dtype=object)

In [29]:
predicted_label = new_data_predictions[0]

print("Predicted Exercise Category:", predicted_label[0])

Predicted Exercise Category: A
